# Search inside astroquery catalog


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab/IN2P3/CNRS
- creation date : 2023/05/17
- last update : 2023-06-02
- FORS2 : https://github.com/ericnuss/fors2
- Result of StarLight : /sps/lsst/groups/photoz/fors2/FORS2/ResStarlight



          /Users/dagoret/MacOSX/GitHub/LSST/FORS2>ls -l
              total 0
              drwxrwsr-x  3 dagoret  staff   96 Aug 21  2018 ResStarlight
              drwxr-xr-x  9 dagoret  staff  288 Dec 19 18:01 fors2

https://astroquery.readthedocs.io/en/latest/

## Import

In [1]:
import os,sys,re
import h5py
from astropy.table import Table
import random

from astropy import coordinates
from astropy import units as u
from astropy.coordinates import Angle
from astroquery.skyview import SkyViewClass
from astroquery.skyview import *
# https://astroquery.readthedocs.io/en/latest/api/astroquery.skyview.SkyViewClass.html

from astropy.wcs import WCS

from astroquery.sdss.core import SDSS
from astropy import coordinates as coords

In [ ]:
# Import some generally useful packages

import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
import pandas as pd
from itertools import cycle, islice


In [ ]:
from scipy import ndimage

#from scipy import interp as scinterp
from scipy.interpolate import interp1d

import glob
import collections
#from def_raw_seds import *

In [ ]:
# to enlarge the sizes
params = {'legend.fontsize': 'large',
          'figure.figsize': (4, 4),
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
import matplotlib.offsetbox
props = dict(boxstyle='round',edgecolor="w",facecolor="w", alpha=0.5)

In [ ]:
def GetColumnHfData(hff,list_of_keys,nameval):
    """
    Extract hff atttribute 
    
    parameters
      hff           : descriptor of h5 file
      list_of_keys : list of exposures
      nameval      : name of the attribute
      
    return
           the array of values in the order of 
    """
    

    all_data = []
    for key in list_of_keys:
        group=hff.get(key)
        val=group.attrs[nameval]
        all_data.append(val)
    return all_data


In [ ]:
def ReadFors2h5FileAttributes(hf):
    hf =  h5py.File(input_file_h5, 'r') 
    list_of_keys = list(hf.keys())
    # pick one key    
    key_sel =  list_of_keys[0]
    # pick one group
    group = hf.get(key_sel)  
    #pickup all attribute names
    all_subgroup_keys = []
    for k in group.attrs.keys():
        all_subgroup_keys.append(k)
    # create info
    df_info = pd.DataFrame()
    for key in all_subgroup_keys:
        arr=GetColumnHfData(hf, list_of_keys ,key)
        df_info[key] = arr
    df_info.sort_values(by="num", ascending=True,inplace=True)
    df_info_num = df_info["num"].values
    key_tags = [ f"SPEC{num}" for num in df_info_num ]
    df_info["name"] = key_tags
    
    #'Nsp', 'RT', 'RV', 'Rmag', 'dec', 'eRV', 'lines', 'num', 'ra', 'redshift',
    
    df_info = df_info[['num' ,'name', 'ra', 'dec', 'Rmag','redshift','lines','RT','RV','eRV','Nsp']]
    return df_info
    

In [ ]:
WL_BALMER_BREAK= 3646.0 
NQUANTILES = 10
quantiles = np.linspace(0,1,NQUANTILES+1)
quantiles

In [ ]:
FLAG_REMOVE_RMAG0 = False

In [ ]:
input_file_h5  = 'data/FORS2spectra.hdf5'

## Start

In [ ]:
cwd=os.path.abspath("")
main_dir = cwd
print(f"notebook current executing path : {cwd}")

## READ file

In [ ]:
hf =  h5py.File(input_file_h5, 'r') 
list_of_keys = list(hf.keys())

In [ ]:
df_info = ReadFors2h5FileAttributes(hf)

In [ ]:
df_info.sort_values(by="num", ascending=True,inplace=True)

In [ ]:
df_info.reset_index(drop=True, inplace=True) 

In [ ]:
df_info_num = df_info["num"].values

In [ ]:
key_tags = [ f"SPEC{num}" for num in df_info_num ]

### Get dataset

- wavelength and flux

In [ ]:
all_df = []   
idx=0
for key in key_tags :
    group = hf.get(key)
    df = pd.DataFrame()
    df["wl"] = np.array(group.get("wl"))
    df["fl"] = np.array(group.get("fl")) 
    all_df.append(df)
    idx+=1

In [ ]:
idx=0
fig = plt.figure(figsize=(10,6))
ax=fig.add_subplot(1,1,1)
ax.plot(all_df[idx]["wl"], all_df[idx]["fl"],'b-')
ax.set_xlabel("$\lambda  (\AA)$ ")
title = f"FORS2 : {key_tags[idx]}"
ax.set_title(title)
ax.set_ylabel("flux (maggies)")

# Plots

## Position of SL galaxies vs redshifts

In [ ]:
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot(1,1,1)
#im = ax.scatter(all_ra,all_dec,c=all_colors2,s=10,marker="o",alpha=0.5,cmap=jet2)
im = ax.scatter(df_info.ra,df_info.dec,c=df_info.redshift,s=10,marker="o",alpha=0.5,cmap="jet")
ax.grid()
ax.set_xlabel("ra (deg)")
ax.set_ylabel("dec (deg)")
ax.set_title("Sky location of FORS2 galaxies")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('redshift')

## Position vs Redshift

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
for idx, ax in enumerate(axes.flatten()):
    if idx == 0:
        ax.hist(df_info.redshift,bins=50,facecolor="b");
        ax.set_xlabel("redshift")
    else:
        im = ax.scatter(df_info.ra,df_info.dec,c=df_info.redshift,s=10,marker="o",alpha=0.5,cmap="jet")
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label('redshift')
        ax.grid()
        ax.set_xlabel("ra (deg)")
        ax.set_ylabel("dec (deg)")
        ax.set_title("Sky location of FORS2 galaxies")
        #fig.colorbar(im, ax=ax)
plt.tight_layout()  
plt.suptitle("redshift",fontsize=20)

In [ ]:
index_selected=np.where(df_info.redshift<1)[0]

In [ ]:
fig = plt.figure(figsize=(5,4))
ax = fig.add_subplot(1,1,1)
#im = ax.scatter(all_ra,all_dec,c=all_colors2,s=10,marker="o",alpha=0.5,cmap=jet2)
im = ax.scatter(df_info.ra.iloc[index_selected],df_info.dec.iloc[index_selected],c=df_info.redshift.iloc[index_selected],s=10,marker="o",alpha=1,cmap="jet")
ax.grid()
ax.set_xlabel("ra (deg)")
ax.set_ylabel("dec (deg)")
ax.set_title("Sky location of FORS2 galaxies")
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('redshift')

# Plot on celestial maps
https://learn.astropy.org/tutorials/plot-catalog.html

In [ ]:
ra_toplot = coordinates.Angle(df_info.ra.iloc[index_selected].values*u.degree)
ra_toplot = ra_toplot.wrap_at(180*u.degree)
dec_toplot = coordinates.Angle(df_info.dec.iloc[index_selected].values*u.degree)
redshifts_toplot = df_info.redshift.iloc[index_selected]

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra_toplot.radian, dec_toplot.radian,c=redshifts_toplot)
ax.grid()

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(ra_toplot.radian, dec_toplot.radian,c=redshifts_toplot)
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)

# Position

In [ ]:
df_info.describe()

In [ ]:
ra_angle = Angle(df_info.ra.mean(), u.deg)
dec_angle =  Angle(df_info.dec.mean(), u.deg)
ra_radius_std = Angle(df_info.ra.std(),u.deg)
dec_radius_std = Angle(df_info.dec.std(),u.deg)
ra_radius_minmax = Angle((df_info.ra.max()-df_info.ra.min())/2.,u.deg)
dec_radius_minmax = Angle((df_info.dec.max()-df_info.dec.min())/2.,u.deg)

In [ ]:
ra_angle

In [ ]:
dec_angle

In [ ]:
ra_radius_minmax.arcmin

In [ ]:
dec_radius_minmax.arcmin

## Position of galaxies vs Rmag

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
for idx, ax in enumerate(axes.flatten()):
    if idx == 0:
        ax.hist(df_info.Rmag,bins=50,facecolor="b");
        ax.set_xlabel("Rmag")
    else:
        im = ax.scatter(df_info.ra,df_info.dec,c=df_info.Rmag,s=10,marker="o",alpha=0.5,cmap="jet")
        cbar = fig.colorbar(im, ax=ax)
        cbar.set_label('Rmag')
        ax.grid()
        ax.set_xlabel("ra (deg)")
        ax.set_ylabel("dec (deg)")
        ax.set_title("Sky location of FORS2 galaxies")
        #fig.colorbar(im, ax=ax)
plt.tight_layout() 
plt.suptitle("Rmag",fontsize=20)

# In SkyView

In [ ]:
SkyView().get_image_list(position='RXJ0054.0-2823',survey=['DSS','GALEX Near UV'])

## DSS

In [ ]:
all_img = SkyViewClass().get_images(position='RXJ0054.0-2823',survey=['DSS'],radius=0.5*u.deg)

In [ ]:
the_img = all_img[0]

In [ ]:
the_img[0].header

In [ ]:
wcs = WCS(the_img[0].header)

In [ ]:
fig = plt.figure(figsize=(10,10))
#fig.set_figheight(15)
#fig.set_figwidth(15)
ax = fig.add_subplot(1,1,1)
ax.imshow(the_img[0].data,origin='lower')
plt.show()

In [ ]:
fig=plt.figure(figsize=(10,10))
ax = plt.subplot(projection=wcs)
ax.imshow(the_img[0].data, origin='lower')
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
ax.set_title("DSS")

In [ ]:
fig=plt.figure(figsize=(10,10))
ax = plt.subplot(projection=wcs)
ax.imshow(the_img[0].data, origin='lower')
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')

ax.scatter(ra_toplot.deg, dec_toplot.deg, transform=ax.get_transform('icrs'),c='r',s=10)
ax.set_title("FORS2 and DSS")

## GALEX

In [ ]:
all_img = SkyViewClass().get_images(position='RXJ0054.0-2823',survey=['GALEX Near UV'],radius=0.5*u.deg)
the_img = all_img[0]

In [ ]:
wcs = WCS(the_img[0].header)

In [ ]:
from astropy.visualization import (MinMaxInterval, SqrtStretch,ImageNormalize,ManualInterval, PercentileInterval, AsymmetricPercentileInterval,ZScaleInterval)
                        
from astropy.visualization.stretch import SinhStretch, LinearStretch,SqrtStretch,LogStretch
interval = ZScaleInterval()
transform = SqrtStretch() + ZScaleInterval()

from matplotlib.colors import LogNorm

norm = ImageNormalize(the_img[0].data, interval = MinMaxInterval(), stretch=LogStretch())


In [ ]:
fig=plt.figure(figsize=(10,10))
ax = plt.subplot(projection=wcs)
ax.imshow(the_img[0].data, origin='lower',norm=norm)
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
ax.set_title("GALEX Near UV")

In [ ]:
fig=plt.figure(figsize=(10,10))
ax = plt.subplot(projection=wcs)
ax.imshow(the_img[0].data, origin='lower',norm=norm)
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
ax.scatter(ra_toplot.deg, dec_toplot.deg, transform=ax.get_transform('icrs'),c='r',s=10)
ax.set_title("FORS2 and GALEX Near UV")

## MAST Missions

In [ ]:
ra_angle

In [ ]:
dec_angle

In [ ]:
from astroquery.mast import MastMissions

missions = MastMissions()

In [ ]:
missions.mission

# Surveys
https://astroquery.readthedocs.io/en/latest/skyview/skyview.html

In [ ]:
SkyView.list_surveys() 

# SDSS

https://astroquery.readthedocs.io/en/latest/sdss/sdss.html

In [ ]:
coords = coordinates.SkyCoord(df_info.ra.mean(),df_info.dec.mean(), unit="deg")  # defaults to ICRS frame

In [ ]:
th1 = (df_info.ra.max()-df_info.ra.min())/2.

In [ ]:
th2 = (df_info.dec.max()-df_info.dec.min())/2.

In [ ]:
max_radius = Angle(max(th1,th2),u.deg)

In [ ]:
max_radius

In [ ]:
result_spectro = SDSS.query_region(coords,radius = max_radius, spectro=True )
results = SDSS.query_region(coords)

In [ ]:
coordinates.SkyCoord('0h8m05.63s +14d50m23.3s')

In [ ]:
results = SDSS.query_region(coordinates.SkyCoord('0h8m05.63s +14d50m23.3s'),spectro=True)

In [ ]:
results

In [ ]:
pos = coordinates.SkyCoord('0h8m05.63s +14d50m23.3s', frame='icrs')
xid = SDSS.query_region(pos, radius='5 arcsec', spectro=True)
print(xid)

In [ ]:
redshift = xid['z'][0]
redshift

In [ ]:
sp = SDSS.get_spectra(matches=xid)
im = SDSS.get_images(matches=xid, band='g')

`sp[0]` stores all the files related to the spectra for the object of interest. This is actually an array of several HDU in the FITS format 

The spectrum is stored as a table in the second item of the list. That means that we can get the Table doing the following

In [ ]:
spectra_data = sp[0][1].data

In [ ]:
spectra_data

In [ ]:
spectra_data.shape

In [ ]:
plt.plot(10**spectra_data['loglam'], spectra_data['flux'])
plt.xlabel('wavelenght (Angstrom)')
plt.ylabel('flux (nanomaggies)')
plt.title('SDSS spectra')

In [ ]:
lines = sp[0][3].data

In [ ]:
lines['LINENAME']

In [ ]:
for n in ['[O_II] 3727', '[O_III] 5007', 'H_alpha']:
    print(n, " ->", lines['LINEWAVE'][lines['LINENAME']==n])

In [ ]:
plt.plot(10**spectra_data['loglam']/(1+redshift), spectra_data['flux'], color='black')
plt.axvline(x=lines['LINEWAVE'][lines['LINENAME']=='[O_II] 3727'], label=r'O[II]', color='blue')
plt.axvline(x=lines['LINEWAVE'][lines['LINENAME']=='[O_III] 5007'], label=r'O[III]', color='red')
plt.axvline(x=lines['LINEWAVE'][lines['LINENAME']=='H_alpha'], label=r'H$\alpha$', color='green')

plt.xlabel('wavelenght (Angstrom)')
plt.ylabel('flux (nanomaggies)')
plt.title('SDSS spectrum')
plt.legend()

In [ ]:
sp[0].info()

In [ ]:
sp[0][2].data

In [ ]:
xid = SDSS.query_region(pos, radius='5 arcmin', spectro=True)

In [ ]:
print(xid)

In [ ]:
sp = SDSS.get_spectra(matches=xid)

In [ ]:
sp[2]